In [ ]:
#@title 設定

# autoreload
%load_ext autoreload
%autoreload 2

# Git LFS & リポジトリのクローン
!apt-get install -qq git-lfs && git lfs install
!git clone --depth 1 https://github.com/khkata/human-in-the-loop-transcribe.git /content/human-in-the-loop-transcribe
!git clone --depth 1 https://huggingface.co/datasets/khkata/human-in-the-loop-transcribe /content/human-in-the-loop-transcribe/data

# 作業ディレクトリへ移動
%cd /content/human-in-the-loop-transcribe
%pip install git+https://github.com/openai/whisper.git

#  依存関係のインストール
%pip install -q -e .

# システム環境パッケージ
!apt-get update -qq && apt-get install -qq mecab libmecab-dev mecab-ipadic-utf8

# config.py
!python -m src.config

In [ ]:
# 特徴量抽出
!python -m src.extract_feats

In [ ]:
#@title 訓練

!python src.train

from src.train import retrain_full_data
retrain_full_data()

In [ ]:
#@title 評価

!python src/evaluate.py \
  --test_features $TEST_FEATURES \
  --scaler_path   $SCALER_PATH \
  --model_path    $MODEL_PATH \
  --processed_csv $TEST_PROCESSED_CSV

In [ ]:
#@title ROVER-3

!python src/integrate.py \
  --human_csv      $TEST_HUMAN_CSV \
  --processed_csv  $TEST_PROCESSED_CSV \
  --integrated_csv $TEST_INTEGRATED_CSV

In [ ]:
#@title Cost-WER Curve

!python src/cost_wer_curve.py \
  --integrated_csv $TEST_INTEGRATED_CSV \
  --human_cost 15.0 \
  --output_dir /content/drive/MyDrive/.../outputs \
  --output_filename cost_wer_curve.png